In [26]:
from iris_client import IrisClient, AsyncIrisClient

base_url = "https://api.iris.dioptra.io"

# PUT YOUR CREDENTIALS HERE (DO NOT COMMIT)
username = ""
password = ""

# Check out https://github.com/dioptra-io/iris-client for more info on IrisClient
with IrisClient(base_url, username, password) as iris:
    me = iris.get("/users/me/services").json()
    measurements = iris.all("/measurements/", params={"tag": "collection:exhaustive", "only_mine": False})

In [27]:
def format_uuid(uuid):
    return uuid.replace("-", "_")

def get_tables(measurement):
    return [
        f"results__{format_uuid(measurement['uuid'])}__{format_uuid(agent['agent_uuid'])}" 
        for agent in measurement["agents"]
    ]

tables = get_tables(measurements[0])

In [28]:
credentials = dict(
    base_url=me["clickhouse"]["base_url"],
    database="iris",
    username=me["clickhouse"]["username"],
    password=me["clickhouse"]["password"]
)

In [29]:
from pych_client import ClickHouseClient

# Check out https://github.com/dioptra-io/pych-client for more info on ClickHouseClient
with ClickHouseClient(**credentials) as client:
    for row in client.iter_json(
        f"SELECT probe_src_addr, probe_dst_prefix, probe_ttl, reply_src_addr FROM {tables[0]} LIMIT 10"
    ):
        print(row)

{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 8, 'reply_src_addr': '::ffff:62.40.98.178'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 11, 'reply_src_addr': '::ffff:141.101.71.2'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 12, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 13, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 14, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 15, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 16, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'prob